## Instagram crawler <br>
download all the images to local.<br>
<br>
#user to download
#user_name = 'vinsfinsmotohama'
#download path
#dname = './instagram_clawler_output/' + user_name + '/'
#total query post = 12*(max_post_iter+1)
#max_post_iter=10

In [2]:
"""
Created on Mon Sep  9 11:47:09 2019

instagram crawler
@author: yoko
"""

import urllib
import requests
import json
import os
import re
import pyquery

all_posts = []
post_cnt=0

def get_web_html(url):
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response.text
        else:    
            print('Wrong：', response.status_code)        
    except Exception as e:
        print(e)
        return None

#only able to query 12 post of one iteration
def get_twelve_post(edges):   
    global all_posts
    global post_cnt
    
    #12 post of this for loop
    for edge in edges:
        shortcode = edge['node']['shortcode']
        url_shortcode = 'https://www.instagram.com/p/'+shortcode+'/?__a=1'
        
        
        
        with urllib.request.urlopen(url_shortcode) as temp_u:
            js_data = json.loads(temp_u.read().decode())
        photo_in_post_cnt=0
        post = []
        if 'edge_sidecar_to_children' in js_data['graphql']['shortcode_media']:
            edges_shortcode = js_data['graphql']['shortcode_media']['edge_sidecar_to_children']['edges']
            #num of pic in single post of this for loop            
            for edge_s in edges_shortcode:
                if edge_s['node']['is_video'] and edge_s['node']['video_url'] != 'https://static.cdninstagram.com/rsrc.php/null.jpg':
                    display_url = edge_s['node']['video_url']
                elif edge_s['node']['display_url']:
                    display_url = edge_s['node']['display_url']
                post.append(display_url)
                photo_in_post_cnt=photo_in_post_cnt+1  
        else:            
            if js_data['graphql']['shortcode_media']['is_video'] and js_data['graphql']['shortcode_media']['video_url'] != 'https://static.cdninstagram.com/rsrc.php/null.jpg':
                display_url = js_data['graphql']['shortcode_media']['video_url']
            elif js_data['graphql']['shortcode_media']['display_url']:
                display_url = js_data['graphql']['shortcode_media']['display_url']
            post.append(display_url)
        all_posts.append(post)
        post_cnt=post_cnt+1

#download all the images to local
def save_from_url_to_local(dname):
    for i in range(0,len(all_posts)):
        for j in range(0,len(all_posts[i])):
            if re.search('\.mp4\?', all_posts[i][j]):
                file_name = dname+str(len(all_posts)-1-i)+'_'+str(j)+'.mp4'
            else:
                file_name = dname+str(len(all_posts)-1-i)+'_'+str(j)+'.png'
            if os.path.isfile(file_name):
                print(file_name," already exist.  No need to download for remaining post.")
                return False
            else:                
                print("saving ",all_posts[i][j]," to ", file_name)
                urllib.request.urlretrieve(all_posts[i][j], file_name)
    print("First time to query this account.  Download all finished.")
    return True
    
#user to download
#user_name = 'emmawatson'
user_name = 'vinsfinsmotohama'
#download path
#dname = 'C:\\Users\\Download\\' + user_name + '\\'
dname = './instagram_clawler_output/' + user_name + '/'
#total query post = 12*(max_post_iter+1)
max_post_iter=10

if not os.path.exists(dname):
    os.makedirs(dname)
else:
    print(dname," already exist")


url = 'https://www.instagram.com/' + user_name
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36',
}
html = get_web_html(url)
user_id = re.findall('profilePage_([0-9]+)', html, re.S)[0]
doc = pyquery.PyQuery(html)
items = doc('script[type="text/javascript"]').items()

print("user_id:",user_id)
print("url:",url)

#query latest 12 post
for item in items:
    if item.text().strip().startswith('window._sharedData'):
        js_data = json.loads(item.text()[21:-1], encoding='utf-8')
        edges = js_data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['edges']
        cursor = js_data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['page_info']['end_cursor']
        flag = js_data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['page_info']['has_next_page']
        get_twelve_post(edges)
            
#query remaining posts other than latest 12  #total query post = 12*(max_post_iter+1)
for index in range(0,max_post_iter):  
    if (flag):
        url_next = 'https://instagram.com/graphql/query/?query_id=17888483320059182&id='+user_id+'&first=12&after='+cursor
        print('next url:',url_next)
        with urllib.request.urlopen(url_next) as temp_u:
            js_data = json.loads(temp_u.read().decode())
        edges = js_data['data']['user']['edge_owner_to_timeline_media']['edges']
        cursor = js_data['data']['user']['edge_owner_to_timeline_media']['page_info']['end_cursor']
        flag = js_data['data']['user']['edge_owner_to_timeline_media']['page_info']['has_next_page']
        get_twelve_post(edges)
    else:
        break
      
save_from_url_to_local(dname)

user_id: 5926259794
url: https://www.instagram.com/vinsfinsmotohama
next url: https://instagram.com/graphql/query/?query_id=17888483320059182&id=5926259794&first=12&after=QVFDSk5OaFFMVG5ZLXdQUkZFd3NsaTJQenhoeWgySUVQWDdkQzhnVUdZcmt5YTdzRFNkV1BRTmNOWkRVeXpIQmlVM1pDay1YUWVvaTYzWEdnLTFna2RPZw==
next url: https://instagram.com/graphql/query/?query_id=17888483320059182&id=5926259794&first=12&after=QVFBMVdIQ3VfOUt3ZW04TVhpeG83VGdZUmtvZlhjdko1S082SVAxQWYwWE1xU3FqVEctMmVFNmlBMkN1WHRtUXNVanRCTU5kZW81bmowZlFiWUVtdmMwMQ==
next url: https://instagram.com/graphql/query/?query_id=17888483320059182&id=5926259794&first=12&after=QVFEa1diTnpLeGRnT1ZWeHJPYnQxX3NyOUVZN18wY3EyT3dkNWhCTmZhMFJHSmdRLUtCSmZUTFo0S1NfalFNR01sOGdmUTlkeDVXd002OXljZXZZcjU0Tg==
next url: https://instagram.com/graphql/query/?query_id=17888483320059182&id=5926259794&first=12&after=QVFCQlBmb2ZrVDZZNU5pWEJ2emVpQ3Q1ZnNkU1lGM291UWxNbHE0RHJFU3pwNlJBVVVWNFU4NlFtelprMkF5Y3RTMFZ3dlZ0MmlYWHNNeXV1TzhydkI0Qg==
saving  https://scontent-mrs2-1.cdni

saving  https://scontent-mrs2-1.cdninstagram.com/vp/042350d6acb9944ba6c0e495f49d35f8/5DFD2AF3/t51.2885-15/e35/50006474_214741829371662_1566449908197368834_n.jpg?_nc_ht=scontent-mrs2-1.cdninstagram.com&_nc_cat=106  to  ./instagram_clawler_output/vinsfinsmotohama/21_0.png
saving  https://scontent-mrs2-1.cdninstagram.com/vp/b5f75be621db0048442fc21754111e06/5E013121/t51.2885-15/e35/49907226_303793426856361_5934315451648622846_n.jpg?_nc_ht=scontent-mrs2-1.cdninstagram.com&_nc_cat=109  to  ./instagram_clawler_output/vinsfinsmotohama/20_0.png
saving  https://scontent-mrs2-1.cdninstagram.com/vp/2b89e6c1d63b3140488a5d70e2b79951/5E0CCAB2/t51.2885-15/e35/50035356_544669366047379_6795600796543076871_n.jpg?_nc_ht=scontent-mrs2-1.cdninstagram.com&_nc_cat=107  to  ./instagram_clawler_output/vinsfinsmotohama/19_0.png
saving  https://scontent-mrs2-1.cdninstagram.com/vp/06984205049c4f3234462b05ca1d103e/5E00C6A4/t51.2885-15/e35/49759371_2572224506340811_8537498368220061719_n.jpg?_nc_ht=scontent-mrs2-1.cd

True

In [1]:
!jupyter nbconvert --to html Instagram_crawler.ipynb

[NbConvertApp] Converting notebook Instagram_crawler.ipynb to html
[NbConvertApp] Writing 296360 bytes to Instagram_crawler.html
